# Tflite Model Creation for GRID LSTM Model

TF Lite can optimize existing models to be less memory and cost-consuming, the ideal situation for using machine learning models on mobile and other Edge Devices.

In [ ]:
import numpy as  np # type: ignore
import pandas as pd # type: ignore
import tensorflow as tf # type: ignore
import math

from sklearn.preprocessing import MinMaxScaler # type: ignore
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error # type: ignore

Now,we have to convert the h5 model to tflite model 

In [ ]:
def convert_weights_to_float32(model):
    for layer in model.layers:
        weights = layer.get_weights()
        if weights:
            weights = [tf.cast(w, dtype=tf.float32) for w in weights]
            layer.set_weights(weights)
    return model
model1 = convert_weights_to_float32("Path_to_h5_Model")

converter = tf.lite.TFLiteConverter.from_keras_model(model1)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]
converter.experimental_enable_resource_variables = True 
converter._experimental_lower_tensor_list_ops = False
converter.experimental_new_converter = True
tflite_model = converter.convert()

with open("PATH_TO_SAVE_THE_MODEL", 'wb') as f:
    f.write(tflite_model)

Now,we have to make Predictions on the Tflite model

In [ ]:
interpreter = tf.lite.Interpreter(model_path="Path_to_the_h5_Model")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
predicted_value = []
true_values = []
for i in range(len(X_short_term_test)):
    input_data_1 = np.expand_dims(X_short_term_test[i], axis=0).astype(np.float32)
    input_data_2 = np.expand_dims(X_historical_test[i], axis=0).astype(np.float32)
    interpreter.set_tensor(input_details[0]['index'], input_data_2)
    interpreter.set_tensor(input_details[1]['index'], input_data_1)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    predicted_value.append(output_data[0][0])
    true_values.append(y_test[i])
predicted_value = np.array(predicted_value).reshape(-1, 1)

tflite_predictions_rescaled = scaler.inverse_transform(predicted_value).flatten()
tflite_y_test_rescaled = scaler.inverse_transform(true_values).flatten()

tflite_min_length = min(len(tflite_predictions_rescaled), len(tflite_y_test_rescaled))
tflite_predictions_rescale = tflite_predictions_rescaled[:tflite_min_length]
tflite_y_test_rescale = tflite_y_test_rescaled[:tflite_min_length]

tflite_results = pd.DataFrame(data={'Predictions': tflite_predictions_rescale, 'Actuals': tflite_y_test_rescale})

print(tflite_results)

# Error Values for the TFLite Model

In [ ]:
rmse = math.sqrt(mean_squared_error(tflite_y_test_rescale, tflite_predictions_rescale))
mae = mean_absolute_error(tflite_y_test_rescale, tflite_predictions_rescale)
mape = mean_absolute_percentage_error(tflite_predictions_rescale, tflite_y_test_rescale)

In [ ]:
print("RMSE:", rmse)
print("MAE:", mae)
print("MAPE:", mape)